Consider a cutting-stock problem with raw width 181 inch and order summary calling for

• 90 finals of width 21$\frac{5}{8}$ inches

• 51 finals of width 20$\frac{1}{2}$ inches

• 45 finals of width 20 inches

• 11 finals of width 17$\frac{1}{4}$ inches

rawwidth = width of uncut raw

Make an initail pattern with at most 8 cuts.

$Pattern_i = min(8,\frac{rawwidth} {finalwidth_i})$ for $i = 1,\dots ,4$.

Make sure that any new pattern has at most 8 cuts.

a = new pattern.

$\sum_{i =1}^4 a_i <= 8$.

In [2]:
from gurobipy import *

finalwidth = [21.625, 20.5, 20, 17.25]
b = [ 90, 51, 45, 11]
rawwidth = 181

finals = range(len(finalwidth))


################# SUBPROBLEM ########################
# Define knapsack subproblem (its objective coefficients will be set each time it is called)
sub = Model()
items = range(len(finalwidth))
a = {}
for i in items :
    a[i] = sub.addVar(vtype = GRB.INTEGER, lb = 0, obj = 0)
sub.update()            
sub.addConstr(quicksum(finalwidth[i]*a[i] for i in items) <= rawwidth)
#there can be at most 8 cuts
sub.addConstr(quicksum(a[i] for i in items) <= 8)
sub.modelSense = GRB.MAXIMIZE


################# (Restricted) MASTER PROBLEM ########################
master = Model()      

# AT is the transpose of the pattern matrix A
AT = []
for i in finals :
    newpattern = [0 for j in finals]
#find an initial solution with a maximum of eight cuts.
    newpattern[i] = min(8,int(rawwidth/finalwidth[i]))
    print(newpattern)
    AT.append(newpattern)
numpat = len(AT[1])

[8, 0, 0, 0]
[0, 8, 0, 0]
[0, 0, 8, 0]
[0, 0, 0, 8]


Solve the LP relaxation.

In [3]:
x = {}
for j in range(numpat) :
    x[j] = master.addVar(lb = 0, obj = 1)

master.update()

cons = {}
for i in finals :
    cons[i] = master.addConstr(quicksum(AT[j][i]*x[j] for j in range(numpat)) == b[i])      


################# LOOP that repeatedly solves the MASTER PROBLEM and SUBPROBLEM ########################
updated = True

while updated :
    master.optimize()
    
    print("\nCurrent solution to the Master Problem has the basic variables set to:")
    for j in range(numpat) :
        if x[j].X > 0.0001 :
            print(x[j].X)
        
    print("Subproblem will be solved with y-values set to:")        
    for i in finals :
        print(-cons[i].Pi)
        a[i].obj = cons[i].Pi             
    print("\n")    

    # solve the subproblem
    sub.optimize()
    if sub.objVal > 1 :
        newpattern = []
        for i in finals :
            newpattern.append(a[i].x)
        print("\nNew pattern generated by subproblem:")            
        print(newpattern)
        print('\n')
        
        # add new pattern to the master problem by adding a new variable and updating its constraints            
        AT.append(newpattern)
        numpat = numpat + 1
        col = Column(coeffs = newpattern, constrs = master.getConstrs()) 
        x[numpat-1] = master.addVar(lb = 0, obj = 1, column = col)
        master.update() 
        
    else :
        updated = False 
        
         
# Print solution
print('\nThe optimal solution to the LP relaxation of the cutting stock problem  is:')         
print('[ Pattern ] Number of Times')         
for j in range(numpat) :
    if x[j].x > 0.0001 :
        print(AT[j],x[j].x)




Optimize a model with 4 rows, 4 columns and 4 nonzeros
Coefficient statistics:
  Matrix range    [8e+00, 8e+00]
  Objective range [1e+00, 1e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+01, 9e+01]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4625000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.462500000e+01

Current solution to the Master Problem has the basic variables set to:
11.25
6.375
5.625
1.375
Subproblem will be solved with y-values set to:
-0.125
-0.125
-0.125
-0.125


Optimize a model with 2 rows, 4 columns and 8 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 2e+01]
  Objective range [1e-01, 1e-01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [8e+00, 2e+02]
Found heuristic solution: objective 1
Presolve removed 2 rows and 4 columns
Presolve time: 0.00s
Preso